In [16]:
import pandas as pd
import numpy as np
import localsolver
import sys
import math

In [17]:
C_C_df = pd.read_csv(r'../Data/C_C.csv')
B_C_df = pd.read_csv(r'../Data/B_C.csv')

In [18]:
C_C_matrix = np.ones((174, 174)) * 10000000
B_C_matrix = np.ones((141, 174)) * 10000000

In [19]:
# c_c_matrix
OriginID = C_C_df['OriginID'].values.tolist()
DestinID = C_C_df['Destinatio'].values.tolist()
Length = C_C_df['Total_Leng'].values.tolist()
for i in range(len(OriginID)):
    C_C_matrix[OriginID[i]-1][DestinID[i]-1] = Length[i]

In [20]:
# b_c_matrix
OriginID = B_C_df['OriginID'].values.tolist()
DestinID = B_C_df['Destinatio'].values.tolist()
Length = B_C_df['Total_Leng'].values.tolist()
for i in range(len(OriginID)):
    B_C_matrix[OriginID[i]-1][DestinID[i]-1] = Length[i]

In [27]:
nb_customers = 174
truck_capacity = 5
dist_matrix_data = C_C_matrix
dist_depot_data = B_C_matrix
demands_data = np.random.randint(0, 2, nb_customers)
nb_trucks = 40
str_time_limit = "20"

In [35]:
with localsolver.LocalSolver() as ls:
    model = ls.model
    customers_sequences = [model.list(nb_customers) for _ in range(nb_trucks)]
    
    quantity = [None] * nb_trucks
    for k in range(nb_trucks):
        quantity[k] = [model.float(0, demands_data[i]) for i in range(nb_customers)]
        
    model.constraint(model.cover(customers_sequences))
    
    dist_matrix = model.array(dist_matrix_data)
    dist_depot = model.array(dist_depot_data)
    
    route_distances = [None] * nb_trucks
    trucks_used = [None] * nb_trucks
    
    for k in range(nb_trucks):
        sequence = customers_sequences[k]
        c = model.count(sequence)
        
        trucks_used[k] = model.count(sequence) > 0
        
        quantity_array = model.array(quantity[k])
        quantity_lambda = model.lambda_function(lambda j: quantity_array[j])
        route_quantity = model.sum(sequence, quantity_lambda)
        model.constraint(route_quantity <= truck_capacity)
        
        dist_lambda = model.lambda_function(lambda i: model.at(dist_matrix, sequence[i - 1], sequence[i]))
        route_distances[k] = model.sum(model.range(1, c), dist_lambda) + model.iif(
                    trucks_used[k],
                    dist_depot[sequence[0]] + dist_depot[sequence[c - 1]], 0)
        
    for i in range(nb_customers):
        # Each customer must receive at least its demand
        quantity_served = model.sum(quantity[k][i] * model.contains(customers_sequences[k], i)
            for k in range(nb_trucks))
        model.constraint(quantity_served >= demands_data[i])
            
    total_distance = model.sum(route_distances)
    
    model.minimize(total_distance)
#     model.close()
    model.close()

    ls.param.time_limit = int(str_time_limit)
    ls.solve()

LSError: ERROR [function ls_close]: Operand 0 of sum#45588 must be boolean, integer, double, collection or interval. Type provided: array.

In [24]:
dist_lambda

In [33]:
total_distance

In [36]:
dist_matrix